<a href="https://colab.research.google.com/github/shashank-m/covid_mining_papers/blob/master/tf_idf_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import inflect
import time
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords 
from sklearn.metrics.pairwise import cosine_similarity
p = inflect. engine()

In [0]:
from google.colab import drive

In [26]:
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import json
import os

filenames=os.listdir("/content/drive/My Drive/covid_nlp/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv")
biorxiv="/content/drive/My Drive/covid_nlp/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/"

In [0]:
all_files = []

for filename in filenames:
    filename = biorxiv + filename
    file = json.load(open(filename, 'rb'))
    all_files.append(file)

In [29]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def remove_punct(s):
  s=re.sub(r'[!.,;:-]',' ',s) 
  s=' '.join(s.split())
  return s
def replace_nums(s):
  numbers_in_string=re.findall(r'[0-9]+',s) # finds all numbers in string  
  for number in numbers_in_string:
    x=p.number_to_words(int(number)) # convert number to word.
    s=re.sub(number,x,s)  
  return s
def tokenise_lower(s):
  tokens=nltk.word_tokenize(s)
  word_tokens= [i.lower() for i in tokens]
  return [w for w in word_tokens if not w in stop_words] # removes stop words.
 
def unique_words_entire_data(data):
  unique=set()
  for i in data.iteritems():
    unique=unique.union(set(i[1]))
  return len(unique)  
def convert_list_string(l):
  return ' '.join(l)

In [31]:
titles_in_biorxiv=[]
for f in all_files:
  t=f['metadata']['title']
  titles_in_biorxiv.append(t)

titles_in_biorxiv=pd.Series(titles_in_biorxiv)

titles_in_biorxiv=titles_in_biorxiv.apply(remove_punct).apply(replace_nums).apply(tokenise_lower)
print("No of unique words in titles of biorxiv dataset = {}".format(unique_words_entire_data(titles_in_biorxiv)))

# tfidf takes inpit as string. Not list. So convert list to string below.
titles_in_biorxiv=titles_in_biorxiv.apply(convert_list_string)

tf_idf_vec=TfidfVectorizer(max_features=3000)

tf_idf=tf_idf_vec.fit_transform(titles_in_biorxiv)

tf_idf_norm = normalize(tf_idf)

tf_idf_array = tf_idf_norm.toarray()

No of unique words in titles of biorxiv dataset = 2876


In [0]:
def top_k_results(query,tf_idf_vec,tf_idf_array,k):
  """
  compute tfidf vector of query.
  find similarity with each row od tf_array.
  sort by highest k similarity scores and return these documents.
  """
  query=query.apply(remove_punct).apply(replace_nums).apply(tokenise_lower).apply(convert_list_string)

  tf_idf_query=normalize(tf_idf_vec.transform(query)).toarray()

  cos_sim=cosine_similarity(tf_idf_query,tf_idf_array)
  
  top_k_documents=np.flip(np.argsort(cos_sim,axis=1),axis=1)[:,:k]

  """
   each row of top_k_documents correspond to each query.
   They contain the indices of the documents.
  """
  return top_k_documents.tolist()
k=3  
query=pd.Series(['Efforts targeted at a universal coronavirus vaccine','Therapeutics research'])
top_k_documents=top_k_results(query,tf_idf_vec,tf_idf_array,k)


In [33]:
for i,q in enumerate(query):
  print('For query "{}"'.format(q))
  print('The top {} documents are :'.format(k))
  print('\n')
  for title in titles_in_biorxiv[top_k_documents[i]]:
    print(title)
  print('*****************') 
  print('\n') 

For query "Efforts targeted at a universal coronavirus vaccine"
The top 3 documents are :


simple model assess wuhan lock effect region efforts covid nineteen epidemic china mainland
ai aided design novel targeted covalent inhibitors sars cov two
den im dengue virus identification shotgun targeted metagenomics
*****************


For query "Therapeutics research"
The top 3 documents are :


efficacy revealed single cell analysis antiviral therapeutics
two systematic review evaluation zika virus forecasting prediction research three public health emergency international concern four five kobres p
article type research article title radar dengue virus infections natural populations aedes aegypti mosquitoes running title dengue virus maintenance mosquito vectors
*****************


